## FPM Reconstruction Script

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import scipy.io as sio
import scipy.linalg as la
import matplotlib.pyplot as plt
import numpy as np
import os, glob, json

# Backend-independent operations
import llops as yp

# FPM functions
from comptic import fpm
from fpm import util

# Generate LED Positions

In [ ]:
# Load LED positions
source_list_mat_file_name = './source_list_quasi_dome.mat'
mat = sio.loadmat(source_list_mat_file_name)
source_list_cart = mat['source_list_cart'].copy()

# Load Data

In [4]:


# Single LED Scan
dataset_path = '/Users/zfphil/Dropbox/Datasets/2017-09-29 Multiplex FPM/fpm_2017_09_29_17_10_41'

# Multiplexed LED Scan
# dataset_path = '/Users/zfphil/Dropbox/Datasets/2017-09-29 Multiplex FPM/fpm_2017_09_29_17_17_47'

mat = sio.loadmat(source_list_mat_file_name)


# Define system parameters for our system
system_parameters = {
                         'objective_numerical_aperture': 0.25,
                         'objective_magnification': 10.0,
                         'system_magnification': 1.0,
                         'camera_pixel_size': 6.5e-6,
                         'camera_is_color': False,
                         'illumination_wavelength': 0.53e-6,
                         'illumination_source_position_list_cartesian': mat['source_list_cart'].copy()
                     }

# Load frames
frame_file_list = sorted(glob.glob(os.path.join(dataset_path, '*.tif')))

# Deal with .tiff stack case
if len(frame_file_list) == 1:
    frame_list = util.readMultiPageTiff(frame_file_list[0])
else:
    frame_list = []
    for frame_file in frame_file_list:
        frame_list.append(util.readMultiPageTiff(frame_file))


# Process Data

In [ ]:
fpm_opt = fpm.FpmOptions()

# Define custom options
fpm_opt.roi = iotools.Roi(x_start=225, x_end=275, y_start=225, y_end=275)
fpm_opt.alg_gd_step_size = 1e-1
fpm_opt.alg_nest_alpha = 1e-3
fpm_opt.alg_nest_beta = 1e-4
fpm_opt.led_auto_calib_use_pre = True
fpm_opt.led_auto_calib_enabled = False
fpm_opt.live_plot = True
fpm_opt.max_na = 0.5
fpm_opt.max_it = 25
fpm_opt.lm_delta2 = 5e0
fpm_opt.algorithm = "seq_gd"
fpm_opt.objective_function_type = 'intensity'

# Create FPM solver object and run solver
fpm_obj = fpm.FpmSolver(dataset=fpm_dataset, options=fpm_opt)

# Run Solver
fpm_obj.run()

# Plot Result
fpm_obj.plotResult()

## Update the LED positions and save

In [ ]:
if True:
    mat['source_list_na'] = fpm_obj.source_list_na.copy()
    sio.savemat(source_list_mat_file_name,mat)